# CH3-4

## TOC<a id='toc'></a>
* [Ch3 Notes](#ch3_notes)
* [Ch4 Notes](#ch4_notes)

### CH3 Notes <a id='ch3_notes'></a>
[toc](#toc)

### Generic Mapping Types
* good chedking if mapping: `isinstance(my_dict, abc.Mapping)`
    - better than just checking for dict to allow other maps
* all mapping types in the standard library use the basic `dict` in their implementation, so they all have to have *hashable* keys
* object is hashable if:
    - it has a hash value which never changes during its lifetime (needs a `__hash__()` method)
    - can be compared to other objects (need an `__eq__()` method)
    - and objects wich compare equal hash equal
* user defined types hash to `id()`, and all compare to not equal. If implement `__eq__()`, then only hashable if all attributes are immutable.

* Use *dictcomps*
* cool methods:
    - `d.get(k, [default])`
    - `d.__missing__(k)` - called when `__getitem__` cannot find key (only in defaultdict)
    - `d.pop(k, [default])`
    - `d.setdefault(k, [default])` - if k in d, return d[k], otherwise set to default and return it
        - very efficient way to handle missing keys (particularly when value is mutable)
    - `d.update(m, [**kwargs])` - m can be mapping or iterable of (key, value) pairs
        * duck typing: first checks if has `keys` method. Otherwise falls back to iterating over m as tuples
        * <font color=red> What is the **kwargs for? </font>

In [10]:
mydict = {1:2}

In [11]:
mydict.update([(3,4)])
mydict

{1: 2, 3: 4}

In [13]:
mydict.update({7:8}, new=10, old=11)
mydict

{1: 2, 3: 4, 7: 8, 'new': 10, 'old': 11}

m

### CH4 Notes <a id='ch4_notes'></a>
[toc](#toc)

[toc](#toc)